In [66]:
pip install sklearn


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [67]:
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.linear_model import LogisticRegression as lr
from sklearn.metrics import make_scorer, accuracy_score, confusion_matrix, classification_report
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE, RandomOverSampler
from sklearn.model_selection import GridSearchCV

In [68]:
#loading the dataset
data=pd.read_csv("TowerDefectSD.csv")

In [69]:
data.head(10)

,Tower_Id,Voltage_Id,Tower_Type,Tower_Config,Fdn_Type,Wind_Zone,Ckts_on_Tower,Vulnerability,Terrain_Type,sd1,sd2,sd3,sd4,sd5
0,130965,4,QA,VERTICAL,FS,4.0,MULTI CIRCUIT,NORMAL,PLAIN,1,0,0,0,0
1,130966,4,DD,VERTICAL,DFR,4.0,DOUBLE CIRCUIT,NORMAL,HILLY,2,0,0,0,3
2,130999,4,DA,VERTICAL,WET,4.0,DOUBLE CIRCUIT,NORMAL,PLAIN,2,0,0,0,0
3,131031,4,QD,VERTICAL,FS,4.0,MULTI CIRCUIT,NORMAL,PLAIN,30,0,0,0,0
4,131032,4,QD,VERTICAL,FS,4.0,MULTI CIRCUIT,NORMAL,PLAIN,1,0,0,0,0
5,131047,4,QB,VERTICAL,FS,4.0,MULTI CIRCUIT,NORMAL,PLAIN,2,0,0,0,0
6,131049,4,QA,VERTICAL,FS,4.0,MULTI CIRCUIT,NORMAL,PLAIN,0,0,0,0,12
7,131050,4,QA,VERTICAL,FS,4.0,MULTI CIRCUIT,NORMAL,PLAIN,0,0,12,0,0
8,131064,4,DA,VERTICAL,NaN,4.0,DOUBLE CIRCUIT,NORMAL,PLAIN,0,0,0,0,0
9,131083,1,DD,HORIZONTAL,WET,4.0,DOUBLE CIRCUIT,NORMAL,PLAIN,0,0,0,0,0


In [70]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163880 entries, 0 to 163879
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Tower_Id       163880 non-null  int64  
 1   Voltage_Id     163880 non-null  int64  
 2   Tower_Type     160999 non-null  object 
 3   Tower_Config   159762 non-null  object 
 4   Fdn_Type       153304 non-null  object 
 5   Wind_Zone      162465 non-null  float64
 6   Ckts_on_Tower  161983 non-null  object 
 7   Vulnerability  163880 non-null  object 
 8   Terrain_Type   159650 non-null  object 
 9   sd1            163880 non-null  int64  
 10  sd2            163880 non-null  int64  
 11  sd3            163880 non-null  int64  
 12  sd4            163880 non-null  int64  
 13  sd5            163880 non-null  int64  
dtypes: float64(1), int64(7), object(6)
memory usage: 17.5+ MB


In [71]:
data.describe()

,Tower_Id,Voltage_Id,Wind_Zone,sd1,sd2,sd3,sd4,sd5
count,163880.000000,163880.000000,162465.000000,163880.000000,163880.000000,163880.000000,163880.000000,163880.000000
mean,338645.817659,3.855321,3.476466,0.426794,0.009415,0.791475,0.013229,0.675842
std,115013.543844,0.436580,1.048098,1.326141,0.194137,2.310146,0.226392,2.035451
min,130962.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,238484.750000,4.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,351448.500000,4.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,442345.250000,4.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,532409.000000,4.000000,7.000000,71.000000,15.000000,76.000000,21.000000,75.000000


In [72]:
col=data.columns         #list of columns

In [73]:
for i in col:
    print(len(data[i].unique()))     #count of unique values in column
    print(data[i].unique())          #list of unique values in a column
    print()

163880
[130965 130966 130999 ... 532392 532407 532409]

4
[4 1 3 2]

169
['QA' 'DD' 'DA' 'QD' 'QB' 'DC' 'DB' 'A' 'QC' 'B' 'C' 'TP' nan ' DD' 'D'
 'SPL C' 'DC+0' 'DA+0' 'DA+3' 'DB+0' 'DD+0' 'DA+6' 'DA+18' 'DB+3' 'DD+6'
 'DC+6' 'DD+18' 'DB+6' 'DC+3' 'DD+9' 'ANCHOR TOWER' 'DCT' 'E' 'DE' 'J'
 'MA' 'MB' 'MD' 'SPL D' 'SPL B' 'SPL A' 'B+0' 'D+3' 'C+0' 'GANTRY' 'D+0'
 'C+3' 'RC' 'D+6' 'B+6' 'A+0' 'C+6' 'D+25' 'DDE+6' 'B+3' 'A+3' 'A+6'
 'C+6(TT)' 'DC+9' 'DD+3' 'DC+03' 'DA+03' 'DD+00' 'DA+00' 'DA+06' 'DC+00'
 'DDE' 'DA+9' 'TPT' 'DA+O' 'QDH' 'QBH+10.5' 'DB+9' 'DB+00' 'DD+09'
 'DC (TAR ROADANP)' 'DDE+0' 'QBH+7.5' 'QDH+16.5' 'DC+0             ( T.P)'
 'DD+0    (RC 1.5) (DE)' 'QDH+7.5' 'DD +0' 'CTP' 'DD+000' 'DB+03' 'DB+06'
 'D+18' 'QDH+0' 'ANCHOR+9' 'SPL' 'DB+0               (RAISE CHIM' 'DB + 0'
 'DB + 9 (+1.5 M RC)' 'S' 'R' 'P' 'Q' 'DD + 0' 'DD+25' 'DC + 0' 'T' 'M'
 'DD + 9' 'CR' 'N+0' 'M+0' 'L+0' 'L+3' 'N+6' 'M+6' 'L+6' 'N+3' 'L' 'N'
 'D+15' 'XX' 'YY' 'XX+3' 'ZZ' 'YY+0' 'DR' 'M+3' 'B+9' 'DD+15' 

# DATA PREPROCESSING

In [74]:
# Count of missing values per column
print(data.isnull().sum())


Tower_Id             0
Voltage_Id           0
Tower_Type        2881
Tower_Config      4118
Fdn_Type         10576
Wind_Zone         1415
Ckts_on_Tower     1897
Vulnerability        0
Terrain_Type      4230
sd1                  0
sd2                  0
sd3                  0
sd4                  0
sd5                  0
dtype: int64


In [75]:
# Percentage of missing values per column
print(data.isnull().mean() * 100)

Tower_Id         0.000000
Voltage_Id       0.000000
Tower_Type       1.757994
Tower_Config     2.512814
Fdn_Type         6.453503
Wind_Zone        0.863437
Ckts_on_Tower    1.157554
Vulnerability    0.000000
Terrain_Type     2.581157
sd1              0.000000
sd2              0.000000
sd3              0.000000
sd4              0.000000
sd5              0.000000
dtype: float64


In [76]:
#filling none values in categorical columns (only categorical columns contain none values in this dataset)

for i in col:
    data[i].fillna(data[i].mode()[0], inplace=True)

In [77]:
# Percentage of missing values per column
print(data.isnull().mean() * 100)

Tower_Id         0.0
Voltage_Id       0.0
Tower_Type       0.0
Tower_Config     0.0
Fdn_Type         0.0
Wind_Zone        0.0
Ckts_on_Tower    0.0
Vulnerability    0.0
Terrain_Type     0.0
sd1              0.0
sd2              0.0
sd3              0.0
sd4              0.0
sd5              0.0
dtype: float64


In [78]:
#if any defects, converting value to 1
data[['sd1', 'sd2','sd3','sd4','sd5']] = data[['sd1', 'sd2','sd3','sd4','sd5']].applymap(lambda x: 1 if x > 0 else x)

In [79]:
#converting categorical labels into numerical format
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
label_encoder= LabelEncoder()
scaler= MinMaxScaler()
for i in col[1:9]:
    data[i] = label_encoder.fit_transform(data[i])

for i in col[9:14]:
     # Reshape to ensure it's a 2D array with a single feature (column)
    scaled_values = scaler.fit_transform(data[i].values.reshape(-1, 1))
    # Assign back to DataFrame
    data[i] = scaled_values

In [80]:
#renaming sd5 as Target
data = data.rename(columns={'sd5': 'Target'})

In [81]:
#splitting training and testing data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data[['sd1','sd2','sd3','sd4']], data['Target'], test_size=0.3,shuffle=True)

# Applying SMOTE

In [82]:
# Apply SMOTE to the training data
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Apply RandomOverSampler to the training data
ros = RandomOverSampler(random_state=42)
X_train_ros, y_train_ros = ros.fit_resample(X_train, y_train)

# GridSearchCV

In [83]:
# GridSearchCV setup for all classifiers
param_grid_lr = {
    'C': [0.01, 0.1, 1, 10, 100],
    'solver': ['liblinear', 'lbfgs']
}



param_grid_rf = {
    'n_estimators': [30, 40, 50, 100],
    'max_depth': [None, 1, 3, 5, 10]
}

param_grid_xgb = {
    'n_estimators': [30, 40, 50, 100],
    'learning_rate': [0.01, 0.1, 0.3],
    'max_depth': [3, 5, 7],
    'scale_pos_weight': [1, 5, 10]
}

param_grid_dt = {
    'max_depth': [None, 1, 3, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

param_grid_knn = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance']
}

param_grid_et = {
    'n_estimators': [30, 40, 50, 100],
    'max_depth': [None, 1, 3, 5, 10]
}

param_grid_lgb = {
    'num_leaves': [20, 30, 50, 100],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [30, 40, 50, 100],
    'boosting_type': ['gbdt'],
    'objective': ['binary'],
    'metric': ['binary_logloss'],
    'feature_fraction': [0.9]
}

In [84]:
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    print(f'Accuracy: {accuracy_score(y_test, y_pred):.4f}')
    print('Confusion Matrix:')
    print(confusion_matrix(y_test, y_pred))
    print('Classification Report:')
    print(classification_report(y_test, y_pred))

In [85]:
# Define a custom scoring function for specificity
def specificity_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return tn / (tn + fp)

# Make scorer using the custom function
specificity_scorer = make_scorer(specificity_score)

# Implementing Logistic Regression Classifier

In [86]:
clf = GridSearchCV(lr(random_state=42), param_grid_lr, cv=5, scoring=specificity_scorer)
clf.fit(X_train_smote, y_train_smote)

GridSearchCV(cv=5, estimator=LogisticRegression(random_state=42),
             param_grid={'C': [0.01, 0.1, 1, 10, 100],
                         'solver': ['liblinear', 'lbfgs']},
             scoring=make_scorer(specificity_score))

In [87]:
print(f"Best parameters for Logistic Regression: {clf.best_params_}")

Best parameters for Logistic Regression: {'C': 0.01, 'solver': 'liblinear'}


# Logistic Regression Metrics

In [88]:
print("Logistic Regression with SMOTE:")
evaluate_model(clf, X_test, y_test)

Logistic Regression with SMOTE:
Accuracy: 0.7685
Confusion Matrix:
[[32137  6069]
 [ 5314  5644]]
Classification Report:
              precision    recall  f1-score   support

         0.0       0.86      0.84      0.85     38206
         1.0       0.48      0.52      0.50     10958

    accuracy                           0.77     49164
   macro avg       0.67      0.68      0.67     49164
weighted avg       0.77      0.77      0.77     49164



# Implementing decision tree classifier using gini

In [89]:
from sklearn.tree import DecisionTreeClassifier

In [90]:
clf_gini = GridSearchCV(DecisionTreeClassifier(criterion='gini', random_state=42, class_weight='balanced'), param_grid_dt, cv=5, scoring=specificity_scorer)
clf_gini.fit(X_train_smote, y_train_smote)

GridSearchCV(cv=5,
             estimator=DecisionTreeClassifier(class_weight='balanced',
                                              random_state=42),
             param_grid={'max_depth': [None, 1, 3, 5, 10],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10]},
             scoring=make_scorer(specificity_score))

In [91]:
print(f"Best parameters for DecisionTree: {clf_gini.best_params_}")

Best parameters for DecisionTree: {'max_depth': 1, 'min_samples_leaf': 1, 'min_samples_split': 2}


# Gini metrics

In [92]:
print("Decision tree gini with SMOTE:")
evaluate_model(clf_gini, X_test, y_test)

Decision tree gini with SMOTE:
Accuracy: 0.7686
Confusion Matrix:
[[32185  6021]
 [ 5355  5603]]
Classification Report:
              precision    recall  f1-score   support

         0.0       0.86      0.84      0.85     38206
         1.0       0.48      0.51      0.50     10958

    accuracy                           0.77     49164
   macro avg       0.67      0.68      0.67     49164
weighted avg       0.77      0.77      0.77     49164



# Implementing KNN Classifier

In [93]:
from sklearn.neighbors import KNeighborsClassifier

In [94]:
clf_knn = GridSearchCV(KNeighborsClassifier(), param_grid_knn, cv=5, scoring=specificity_scorer)
  
clf_knn.fit(X_train_smote, y_train_smote) 

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [3, 5, 7, 9],
                         'weights': ['uniform', 'distance']},
             scoring=make_scorer(specificity_score))

In [95]:
print(f"Best parameters for KNeighbors: {clf_knn.best_params_}")

Best parameters for KNeighbors: {'n_neighbors': 9, 'weights': 'distance'}


# KNN metrics

In [96]:
print("KNN with SMOTE:")
evaluate_model(clf_knn, X_test, y_test)

KNN with SMOTE:
Accuracy: 0.7868
Confusion Matrix:
[[36365  1841]
 [ 8641  2317]]
Classification Report:
              precision    recall  f1-score   support

         0.0       0.81      0.95      0.87     38206
         1.0       0.56      0.21      0.31     10958

    accuracy                           0.79     49164
   macro avg       0.68      0.58      0.59     49164
weighted avg       0.75      0.79      0.75     49164



# Implementing Random Forest Classifier

In [97]:
from sklearn.ensemble import RandomForestClassifier

In [98]:
clf_rf = GridSearchCV(RandomForestClassifier(random_state=42), param_grid_rf, cv=5, scoring=specificity_scorer)
clf_rf.fit(X_train_smote,y_train_smote)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
             param_grid={'max_depth': [None, 1, 3, 5, 10],
                         'n_estimators': [30, 40, 50, 100]},
             scoring=make_scorer(specificity_score))

In [99]:
print(f"Best parameters for RandomForest: {clf_rf.best_params_}")

Best parameters for RandomForest: {'max_depth': 1, 'n_estimators': 50}


# RFC metrics

In [100]:
print("RFC with SMOTE:")
evaluate_model(clf_rf, X_test, y_test)

RFC with SMOTE:
Accuracy: 0.7687
Confusion Matrix:
[[32177  6029]
 [ 5342  5616]]
Classification Report:
              precision    recall  f1-score   support

         0.0       0.86      0.84      0.85     38206
         1.0       0.48      0.51      0.50     10958

    accuracy                           0.77     49164
   macro avg       0.67      0.68      0.67     49164
weighted avg       0.77      0.77      0.77     49164



# Implementing ExtraTrees Classifier

In [101]:
from sklearn.ensemble import ExtraTreesClassifier

In [102]:
clf_et = GridSearchCV(ExtraTreesClassifier(random_state=42), param_grid_et, cv=5, scoring=specificity_scorer)
clf_et.fit(X_train_smote,y_train_smote)

GridSearchCV(cv=5, estimator=ExtraTreesClassifier(random_state=42),
             param_grid={'max_depth': [None, 1, 3, 5, 10],
                         'n_estimators': [30, 40, 50, 100]},
             scoring=make_scorer(specificity_score))

In [103]:
print(f"Best parameters for ExtraTrees: {clf_et.best_params_}")

Best parameters for ExtraTrees: {'max_depth': 1, 'n_estimators': 30}


# ETC Metrics

In [104]:
print("ETC with SMOTE:")
evaluate_model(clf_et, X_test, y_test)

ETC with SMOTE:
Accuracy: 0.7687
Confusion Matrix:
[[32177  6029]
 [ 5342  5616]]
Classification Report:
              precision    recall  f1-score   support

         0.0       0.86      0.84      0.85     38206
         1.0       0.48      0.51      0.50     10958

    accuracy                           0.77     49164
   macro avg       0.67      0.68      0.67     49164
weighted avg       0.77      0.77      0.77     49164



# Implementing ADABOOST Classifier

In [105]:
from sklearn.ensemble import AdaBoostClassifier

In [106]:
param_grid_ada = {
    'n_estimators': [30, 40, 50, 100],
    'learning_rate': [0.01, 0.1, 1]
}

In [107]:
base_learner = DecisionTreeClassifier(max_depth=1)

ada_boost = GridSearchCV(AdaBoostClassifier(random_state=42), param_grid_ada, cv=5, scoring=specificity_scorer)
ada_boost.fit(X_train_smote, y_train_smote)

GridSearchCV(cv=5, estimator=AdaBoostClassifier(random_state=42),
             param_grid={'learning_rate': [0.01, 0.1, 1],
                         'n_estimators': [30, 40, 50, 100]},
             scoring=make_scorer(specificity_score))

In [108]:
print(f"Best parameters for ADABoost: {ada_boost.best_params_}")

Best parameters for ADABoost: {'learning_rate': 0.01, 'n_estimators': 30}


# ADABOOST Metrics

In [109]:
print("ADB(DT) with SMOTE:")
evaluate_model(ada_boost, X_test, y_test)

ADB(DT) with SMOTE:
Accuracy: 0.7686
Confusion Matrix:
[[32185  6021]
 [ 5355  5603]]
Classification Report:
              precision    recall  f1-score   support

         0.0       0.86      0.84      0.85     38206
         1.0       0.48      0.51      0.50     10958

    accuracy                           0.77     49164
   macro avg       0.67      0.68      0.67     49164
weighted avg       0.77      0.77      0.77     49164



# Implementing XGBOOST Classifier

In [110]:
from xgboost import XGBClassifier

In [111]:
clf_xgb = GridSearchCV(XGBClassifier(random_state=42), param_grid_xgb, cv=5, scoring=specificity_scorer)
clf_xgb.fit(X_train_smote, y_train_smote)

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=42, ...),
             param_grid={'learning_rate': [0.01, 0.1, 0.3],
                         'max_depth': [3, 5, 7],
                         'n_estimators': [30, 40, 50, 100],
                         'scale_pos_weight': [1, 5, 10]},
             scoring=make_scorer(specificity_score))

In [112]:
print(f"Best parameters for XGBoost: {clf_xgb.best_params_}")

Best parameters for XGBoost: {'learning_rate': 0.3, 'max_depth': 3, 'n_estimators': 100, 'scale_pos_weight': 1}


# XGB Metrics

In [113]:
print("XGB with SMOTE:")
evaluate_model(clf_xgb, X_test, y_test)

XGB with SMOTE:
Accuracy: 0.7677
Confusion Matrix:
[[32077  6129]
 [ 5292  5666]]
Classification Report:
              precision    recall  f1-score   support

         0.0       0.86      0.84      0.85     38206
         1.0       0.48      0.52      0.50     10958

    accuracy                           0.77     49164
   macro avg       0.67      0.68      0.67     49164
weighted avg       0.77      0.77      0.77     49164



# Implementing LightGBM

In [114]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [115]:
import lightgbm as lgb

In [116]:
# Initialize the LightGBM classifier
lgbm = lgb.LGBMClassifier()

# Initialize GridSearchCV
grid_lgbm = GridSearchCV(estimator=lgbm, param_grid=param_grid_lgb, cv=5, scoring=specificity_scorer)

# Fit GridSearchCV
grid_lgbm.fit(X_train_smote, y_train_smote)

[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Info] Number of positive: 71351, number of negative: 71351
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002816 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8
[LightGBM] [Info] Number of data points in the train set: 142702, number of used features: 4
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warn

GridSearchCV(cv=5, estimator=LGBMClassifier(),
             param_grid={'boosting_type': ['gbdt'], 'feature_fraction': [0.9],
                         'learning_rate': [0.01, 0.05, 0.1],
                         'metric': ['binary_logloss'],
                         'n_estimators': [30, 40, 50, 100],
                         'num_leaves': [20, 30, 50, 100],
                         'objective': ['binary']},
             scoring=make_scorer(specificity_score))

In [117]:
print("Best parameters for LightGBM:", grid_lgbm.best_params_)
best_lgbm = grid_lgbm.best_estimator_

Best parameters for LightGBM: {'boosting_type': 'gbdt', 'feature_fraction': 0.9, 'learning_rate': 0.1, 'metric': 'binary_logloss', 'n_estimators': 100, 'num_leaves': 20, 'objective': 'binary'}


# LGBM Metrics

In [118]:
# Evaluate the best LightGBM model
print("LightGBM with GridSearchCV:")
evaluate_model(best_lgbm, X_test, y_test)

LightGBM with GridSearchCV:
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
Accuracy: 0.7677
Confusion Matrix:
[[32077  6129]
 [ 5290  5668]]
Classification Report:
              precision    recall  f1-score   support

         0.0       0.86      0.84      0.85     38206
         1.0       0.48      0.52      0.50     10958

    accuracy                           0.77     49164
   macro avg       0.67      0.68      0.67     49164
weighted avg       0.77      0.77      0.77     49164



# Adaboost with RFC

In [127]:
base_learner = RandomForestClassifier(random_state=42, max_depth=1, n_estimators=50)

ada_boost_rfc = AdaBoostClassifier(estimator=base_learner, random_state=42, learning_rate=0.01, n_estimators=30)
ada_boost_rfc.fit(X_train_smote, y_train_smote)



AdaBoostClassifier(estimator=RandomForestClassifier(max_depth=1,
                                                    n_estimators=50,
                                                    random_state=42),
                   learning_rate=0.01, n_estimators=30, random_state=42)

# ADB with RFC base : Metrics

In [128]:
print("ADB(RFC) with SMOTE:")
evaluate_model(ada_boost_rfc, X_test, y_test)

ADB(RFC) with SMOTE:
Accuracy: 0.7687
Confusion Matrix:
[[32177  6029]
 [ 5342  5616]]
Classification Report:
              precision    recall  f1-score   support

         0.0       0.86      0.84      0.85     38206
         1.0       0.48      0.51      0.50     10958

    accuracy                           0.77     49164
   macro avg       0.67      0.68      0.67     49164
weighted avg       0.77      0.77      0.77     49164



# Adaboost with ETC

In [129]:
base_learner = ExtraTreesClassifier(random_state=42, max_depth=1, n_estimators=30) 

ada_boost_etc = AdaBoostClassifier(random_state=42, estimator=base_learner, learning_rate=0.01, n_estimators=30 )
ada_boost_etc.fit(X_train_smote, y_train_smote)

AdaBoostClassifier(estimator=ExtraTreesClassifier(max_depth=1, n_estimators=30,
                                                  random_state=42),
                   learning_rate=0.01, n_estimators=30, random_state=42)

# ADB with ETC : Metrics

In [130]:
print("ADB(ETC) with SMOTE:")
evaluate_model(ada_boost_etc, X_test, y_test)

ADB(ETC) with SMOTE:
Accuracy: 0.7687
Confusion Matrix:
[[32177  6029]
 [ 5342  5616]]
Classification Report:
              precision    recall  f1-score   support

         0.0       0.86      0.84      0.85     38206
         1.0       0.48      0.51      0.50     10958

    accuracy                           0.77     49164
   macro avg       0.67      0.68      0.67     49164
weighted avg       0.77      0.77      0.77     49164

